In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from lsst.daf.butler import Butler
from lsst.obs.lsst import LsstCam
from lsst.summit.utils.plotting import plot
from scipy.ndimage import center_of_mass

In [ ]:
butler = Butler('LSSTCam', collections=['LSSTCam/raw/all', 
                                            'LSSTCam/calib/unbounded',
                                             'LSSTCam/raw/guider'])
instrument = 'LSSTCam'
camera = LsstCam.getCamera()

In [ ]:
ampDict = {'Segment00':'C00', 'Segment01':'C01', 'Segment02':'C02', 'Segment03':'C03', 
           'Segment04':'C04', 'Segment05':'C05', 'Segment06':'C06', 'Segment07':'C07', 
           'Segment08':'C17', 'Segment09':'C16', 'Segment10':'C15', 'Segment11':'C14', 
           'Segment12':'C13', 'Segment13':'C12', 'Segment14':'C11', 'Segment15':'C10', }


expId  = 2025102600225
detName = 'R44_SG0'
stampNum = 18 # This is 1 less than the RubinTV movie
detector = camera[detName]
raw = butler.get("guider_raw", exposure=expId, detector=detector.getId(), instrument='LSSTCam')
mdata = raw.metadata
cols = mdata['ROICOLS']
rows = mdata['ROIROWS']
llX = mdata['ROICOL']
llY = mdata['ROIROW']
urX = llX + mdata['ROICOLS']
urY = llY + mdata['ROIROWS']
stamp_ampName = ampDict[mdata['ROISEG']]
print(stamp_ampName)

arr = raw[stampNum].stamp_im.image.array
arr -= np.median(arr)
print(np.min(arr), np.max(arr), np.median(arr))


In [ ]:
%matplotlib inline
fig = plt.figure(figsize=(8,8))
cut = arr[315:335, 65:85]
cut = np.fliplr(cut)
cY, cX = center_of_mass(cut)
print(cY, cX)
vmin, vmax = np.nanpercentile(cut, [50, 99])
plt.imshow(cut, cmap='gray_r', vmin=vmin, vmax=vmax, interpolation='nearest', origin='lower')
plt.scatter([cX], [cY], marker='o', color='red')
plt.savefig(f"/home/cslage/DATA/Stamp_2025102600225_Stamp_R44_SG0_Stamp_19.png")

In [ ]:
cXs = []
cYs = []
stack = np.zeros([20,20])
for stampNum in range(1,50):
    arr = raw[stampNum].stamp_im.image.array
    #arr -= np.median(arr)
    cut = arr[135:155, 206:226]
    cut = np.fliplr(cut)
    stack += np.array(cut)
    cY, cX = center_of_mass(cut)
    cXs.append(cX)
    cYs.append(cY)
cXs = np.array(cXs)
cXs -= np.median(cXs)
cYs = np.array(cYs)
cYs -= np.median(cYs)


In [ ]:
plt.plot(cXs, marker='x')
plt.ylim(-3,3)

In [ ]:
cY, cX = center_of_mass(stack)
print(cY, cX)
vmin, vmax = np.nanpercentile(stack, [50, 99])
plt.imshow(stack, cmap='gray_r', vmin=vmin, vmax=vmax, interpolation='nearest')
plt.scatter([cX], [cY], marker='o', color='red')
